# Example 6.4 Logical Operators for Reed Muller and Hypercube Codes
This notebook illustrates the application of the logical operator algorithm for Reed Muller and Hypercube codes. The example illustrates that Reed Muller codes on $2^t-1$ qubits can be viewed as precision $N = 2^t$ codes. By increasing $t$, we can obtain a logical $\sqrt{P}$ operator which is at the $(t-1)$ st level of the Clifford Hierarchy.

Similarly, for Hypercube codes of dimension $D$, the XP code of precision $N = 2^D$  has a logical generalised controlled Z operator at the $(D-1)$ st level of the Clifford Hierarchy.

This code is made available subject to [GPL licensing](https://www.gnu.org/licenses/gpl-3.0.en.html). Readers who wish to modify the code can either download the [Github repository](https://github.com/m-webster/XPFpackage) or use online services such as [Binder](https://mybinder.org/) or [Colab](https://colab.research.google.com/).

## Reed Muller Codes
The parameters for Reed Muller codes can be varied and are as follows:
- $m > 0$ determines the number of qubits - without punctures, the code is on $2^m$ qubits
- $r \le m$ determines the number of X generators - $|S_X| = \sum_{0 \le i \le r} \binom{m}{i}$
- $p$ is the number of punctures and influences the number of logical qubits.

A Reed Muller code is generated as a CSS code based on the parameters. The logical X and Z operators are then generated. The natural precision of the code is then calculated by looking at the maximum precision at which the code is self-dual.

We then change the precision of the code and determine the logical operators. In general, new diagonal logical operators are found which may be higher in the Clifford heirarchy. We verify that each each of the operators is a logical operator and display the $f$ vector which gives the phase applied to each codeword (ie the logical action).

In [13]:
import add_parent_dir
from reedmuller import *
from hypercube import *
from common import *
from XPAlgebra import *

## Reed Muller parameters
## m determines the number of qubits - without punctures, the code is on 2^m qubits
m = 4
## r determines the number of X generators - |SX| = \sum_{0 \le i le r} \binom{m}{i}
r = 1
# p is the number of punctures - remove the first p columns and rows
p = 1
## set debugging level
setVerbose(False)


def codeSummary(G,N,printCodewords=False):
    ## make an XP code
    C = Code(G,N)
    ## logical identity generators
    LI = getVal(C,'LI')
    ## non-trivial logical operator generators
    LO = getVal(C,'LO')
    print(f'Logical Operators: Precision N={N}')
    print('The f-vector of a logical operator is the phase applied to each codeword')
    if len(LO) > 0:
        for A in LO:
            ## the f-vector is the phase applied to each codeword
            f = ZMat2str(C.Fvector(A),max(11,2*N))
            ## isLO verfies that the operator is a valid logical operator
            status = ('f-vector ' + str(f)) if isLO(A,LI,N) else 'Not LO'
            print(XP2Str(A,N),status)
        # print('\nQubit numbering representation')
        # print(XP2StrAlt(LO,N),"\n")
    else:
        print('None')
    if printCodewords:
        CW,QIndex,LIndex = getVals(C,['Codewords','QIndex','LIndex'])
        print('\n')
        print('Codewords:')
        for i in range(len(CW)):
            print(ZMat2str(LIndex[i]),State2Str(CW[i],N))
    print('\n')


if checkParameters(m,r,p):
    print(f'Reed Muller code with parameters m={m}, r={r}, p={p}')
    print('\n')
    ## We are building a CSS code:
    ## X components of non-diagonal generators
    SXx = RM(r,m,p)
    ## Z components of diag generators - this is dual to SXx
    SZz = RM(m-r-1,m,p)
    ## set precision
    N = 2
    ## make generators
    SX = makeXP(0,SXx,0)
    SZ = makeXP(0,0,SZz)
    G = np.vstack([SX,SZ])

    # print('Stabilizer Generators - qubit numbering representation')
    # print(XP2StrAlt(G,N),"\n")
    print('Stabilizer Generators - vector representation')
    print(XP2Str(G,N),"\n")
    

    codeSummary(G,N)
    P = self_dual(SXx)
    if P > 1:
        N2 = P*2
        print('Natural precision N =',N2)
        G = XPSetN(G,N,N2)
        codeSummary(G,N2)

Maximum value of p = 5
Reed Muller code with parameters m=4, r=1, p=1


Stabilizer Generators - vector representation
XP_2(0|100011100011101|000000000000000)
XP_2(0|010010011011011|000000000000000)
XP_2(0|001001010110111|000000000000000)
XP_2(0|000100101101111|000000000000000)
XP_2(0|000000000000000|100011100011101)
XP_2(0|000000000000000|010010011011011)
XP_2(0|000000000000000|001001010110111)
XP_2(0|000000000000000|000100101101111)
XP_2(0|000000000000000|000010000011001)
XP_2(0|000000000000000|000001000010101)
XP_2(0|000000000000000|000000100001101)
XP_2(0|000000000000000|000000010010011)
XP_2(0|000000000000000|000000001001011)
XP_2(0|000000000000000|000000000100111) 

Logical Operators: Precision N=2
The f-vector of a logical operator is the phase applied to each codeword
XP_2(0|000011111100001|000000000000000) f-vector  0  0
XP_2(0|000000000000000|000000000011111) f-vector  0  2


Natural precision N = 8
Logical Operators: Precision N=8
The f-vector of a logical operator is the pha

## Hypercube Codes
To construct a Hypercube Code, we first construct a $D$-dimensional hypercube. We place qubits on each vertex. 

The stabilizer generators of the code are as follows:
- Diagonal Stabilizers: Z operators on vertices of each 2-dimensional face
- Non-diagonal Stabilizers: X operators on all vertices of the hypercube

For Hypercube codes of dimension $D$, the XP code of precision $N = 2^D$  has a logical generalised controlled Z operator at the $(D-1)$ st level of the Clifford Hierarchy.

In the code below, you can vary the dimension $D$ of the hypercube, as well as the dimension of the faces, xDim and zDim, for construction of the stabilizer generators. The program determines which logical actions can be applied by diagonal logical operators (FD) and the XP operators which apply these actions (LD).

In [14]:

# setVerbose(True)
## D: dimension of Hypercube
D = 3
print(f'{D}-Dimensional Hypercube Code')

## n: number of qubits
n = 1 << D
C = HyperCube(D)

zDim = 2
xDim = D
print(f'Diagonal Stabilizer Generators: Z operators on vertices of each {zDim}-dimensional face')
print(f'Non-Diagonal Stabilizer Generators: X operators on vertices of each {xDim}-dimensional face')
## X stabilizers are on faces of dimension xDim
SX = makeXP(0,C[xDim],0) if xDim >= 0 and xDim <=D else ZMat([],2*n+1)
## Z stabilizers are on face of dimension zDim
SZ = makeXP(0,0,C[zDim]) if zDim >= 0 and zDim <=D else ZMat([],2*n+1)

G = np.vstack([SX,SZ])
# print(XP2Str(G,N))
## Operators are initially precision N=2, but need to convert to precision P
N = 2
P = max(2,n)
G = XPSetN(G,N,P)
N = P

print('\nGenerators G:')
print(XP2Str(G,P))
C = Code(G,P)

Em = getVal(C,'Em')
print('Codespace dimension',len(Em))

LD,FD = getVals(C,['LD','FD'])
print('\nFD: Diagonal logical actions:')
print(ZmatPrint(FD,2*P))
print('\nLD: Logical operators applying these actions:')
print(XP2Str(LD,P))


3-Dimensional Hypercube Code
Diagonal Stabilizer Generators: Z operators on vertices of each 2-dimensional face
Non-Diagonal Stabilizer Generators: X operators on vertices of each 3-dimensional face

Generators G:
XP_8( 0|11111111|00000000)
XP_8( 0|00000000|44440000)
XP_8( 0|00000000|00004444)
XP_8( 0|00000000|44004400)
XP_8( 0|00000000|00440044)
XP_8( 0|00000000|40404040)
XP_8( 0|00000000|04040404)
Codespace dimension 8

FD: Diagonal logical actions:
 1  1  1  1  1  1  1  1
 0  8  0  0  0  0  0  0
 0  0  8  0  0  0  0  0
 0  0  0  8  0  0  0  0
 0  0  0  0  8  0  0  0
 0  0  0  0  0  8  0  0
 0  0  0  0  0  0  8  0
 0  0  0  0  0  0  0  8

LD: Logical operators applying these actions:
XP_8( 1|00000000|00000000)
XP_8( 0|00000000|75753131)
XP_8( 0|00000000|11771177)
XP_8( 0|00000000|13315775)
XP_8( 0|00000000|11553377)
XP_8( 0|00000000|13137575)
XP_8( 0|00000000|77115533)
XP_8( 0|00000000|75571331)
